# Searching for the optimal parameters

In [1]:
#!conda install -y xgboost numpy pandas scipy

In [2]:
#pip install -U joblib

In [3]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn import metrics
import numpy as np
import pandas as pd
import pickle
from sklearn.datasets import fetch_openml

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [5]:
#import the mnist dataset
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [6]:
#separate features and target
X, y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


In [7]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                       test_size=0.1, 
                                       random_state=42)

In [8]:
X_train.shape

(63000, 784)

## Data Augmentation

In [9]:
from scipy.ndimage import shift
X_aug_down = shift(np.array(X_train).reshape(63000,28,28), [0,1,0], cval=0)
X_aug_down.shape

(63000, 28, 28)

In [10]:
X_aug_up = shift(np.array(X_train).reshape(63000,28,28), [0,-1,0], cval=0)
X_aug_up.shape

(63000, 28, 28)

In [11]:
X_aug_right = shift(np.array(X_train).reshape(63000,28,28), [0,0,1], cval=0)
X_aug_right.shape

(63000, 28, 28)

In [12]:
X_aug_left = shift(np.array(X_train).reshape(63000,28,28), [0,0,-1], cval=0)
X_aug_left.shape

(63000, 28, 28)

In [13]:
X_temp = np.concatenate((X_aug_down, X_aug_up, X_aug_right, X_aug_left))
X_temp.shape

(252000, 28, 28)

In [14]:
# Augmented training dataset generated and stored in X_aug and y_aug.
X_aug = np.concatenate((X_train, X_temp.reshape(252000, 784)))
X_aug.shape

(315000, 784)

In [15]:
y_aug = np.concatenate((y_train, y_train, y_train, y_train, y_train))
y_aug.shape

(315000,)

In [16]:
X_train = X_aug
y_train = y_aug


print(len(X_train))
print(len(y_train))

315000
315000


## Preprocessing

In [17]:
# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.transform(X_test.astype(np.float64))

#X_train_scaled = X_train
#X_test_scaled = X_test

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


## XG Boost

There are in general two ways that you can control overfitting in XGBoost:

- The first way is to directly control model complexity.

    - This includes max_depth, min_child_weight and gamma.

- The second way is to add randomness to make training robust to noise.

    - This includes subsample and colsample_bytree.

    - You can also reduce stepsize eta. Remember to increase num_round when you do so.

[source](https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html#:~:text=There%20are%20in,you%20do%20so.)

In [18]:
# modeling: XGBoost
#model = XGBClassifier()

Gridsearch suggestions:  
https://towardsdatascience.com/doing-xgboost-hyper-parameter-tuning-the-smart-way-part-1-of-2-f6d255a45dde

Using this example but simplifying the grid to conserve time.


In [19]:
# define your parameters 

param = {
    "learning_rate": 0.30,
    "max_depth": 15,
    "min_child_weight":1,
    "gamma": 0.0,
    "colsample_bytree": 0.3 
}

In [20]:
# establish cross-validation and gridsearch 
# note: setting cross-validation =2 to save time
# xgb has cross-validation built in so repeating here is unnecessary

#xgb_grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='roc_auc')
xgb_model = XGBClassifier(**param)

In [21]:
# conduct the search (this will take a while)
xgb_model.fit(X_train_scaled, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:23:39] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3,
              enable_categorical=False, gamma=0.0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
# see the best parameters and their score
# print("Best parameters:")
# print(xgb_grid.best_params_)
# print("Best score in grid search:")
# print(xgb_model.best_score_)
# print("best model from grid search:")
# print(xgb_model.score(X_test_scaled, y_test))

Best score in grid search:


AttributeError: 'XGBClassifier' object has no attribute 'best_score_'

In [23]:
# predict
y_preds=xgb_model.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']
['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']


In [24]:
# evaluate
print('Accuracy:', metrics.accuracy_score(y_test, y_preds))
print('Precision:', metrics.precision_score(y_test, y_preds,average='macro'))
print('Recall:', metrics.recall_score(y_test, y_preds,average='macro'))
print('F1 Score:', metrics.f1_score(y_test, y_preds,average='macro'))

Accuracy: 0.9851428571428571
Precision: 0.9849865193732226
Recall: 0.9852401268810149
F1 Score: 0.9850931222167884


## Pickle the model

In [25]:
# xgboost
f = open('xgb_model6.pkl', 'wb')
pickle.dump(xgb_model, f)
f.close() 

## END